In [ ]:
!ls /content/*.zip
!unzip -q /content/train.zip -d /content/train
!unzip -q /content/val.zip   -d /content/val
!unzip -q /content/test.zip  -d /content/test

In [ ]:
!pip install imagecodecs

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, optimizers, callbacks

TRAIN_DIR = '/content/train/train'
VAL_DIR   = '/content/val/val'

IMG_SIZE   = 128
BATCH_SIZE = 32
LR         = 1e-4
EPOCHS     = 50

smooth = 1.0
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    inter = K.sum(y_true_f * y_pred_f)
    return (2. * inter + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
def load_split(folder):
    """Loads all .tif images and their _mask.tif from `folder`, resizes to IMG_SIZE."""
    fns = sorted(f for f in os.listdir(folder) if f.endswith('.tif') and '_mask' not in f)
    N = len(fns)
    X = np.zeros((N, IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
    Y = np.zeros((N, IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
    for i, fn in enumerate(tqdm(fns, desc=f'Loading {folder}')):
        img = (Image.open(os.path.join(folder, fn))
                  .convert('L')
                  .resize((IMG_SIZE,IMG_SIZE), Image.BILINEAR))
        msk = (Image.open(os.path.join(folder, fn.replace('.tif','_mask.tif')))
                  .convert('L')
                  .resize((IMG_SIZE,IMG_SIZE), Image.NEAREST))
        X[i,...,0] = np.array(img)/255.0
        Y[i,...,0] = (np.array(msk)>0).astype(np.float32)
    return X, Y

X_train, y_train = load_split(TRAIN_DIR)
X_val,   y_val   = load_split(VAL_DIR)
print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f" Val shapes: {X_val.shape},   {y_val.shape}")

In [ ]:
def build_unet(input_size=(IMG_SIZE,IMG_SIZE,1)):
    inp = layers.Input(input_size)
    def conv_block(x, f):
        x = layers.Conv2D(f,3,padding='same',activation='relu')(x)
        x = layers.Conv2D(f,3,padding='same',activation='relu')(x)
        return x

    c1 = conv_block(inp, 64); p1 = layers.MaxPooling2D()(c1)
    c2 = conv_block(p1,128); p2 = layers.MaxPooling2D()(c2)
    c3 = conv_block(p2,256); p3 = layers.MaxPooling2D()(c3)
    c4 = conv_block(p3,512); p4 = layers.MaxPooling2D()(c4)

    c5 = conv_block(p4,1024)

    u6 = layers.Conv2DTranspose(512,2,strides=2,padding='same')(c5)
    u6 = layers.concatenate([u6,c4]); c6 = conv_block(u6,512)
    u7 = layers.Conv2DTranspose(256,2,strides=2,padding='same')(c6)
    u7 = layers.concatenate([u7,c3]); c7 = conv_block(u7,256)
    u8 = layers.Conv2DTranspose(128,2,strides=2,padding='same')(c7)
    u8 = layers.concatenate([u8,c2]); c8 = conv_block(u8,128)
    u9 = layers.Conv2DTranspose(64,2,strides=2,padding='same')(c8)
    u9 = layers.concatenate([u9,c1]); c9 = conv_block(u9,64)

    out = layers.Conv2D(1,1,activation='sigmoid')(c9)
    return models.Model(inp,out)

model = build_unet()
model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(LR),
    loss=dice_loss,
    metrics=[dice_coef, 'binary_accuracy']
)

chkpt = callbacks.ModelCheckpoint(
    'best_unet.h5',
    monitor='val_dice_coef',
    mode='max',
    save_best_only=True,
    verbose=1
)
csvlog = callbacks.CSVLogger('training.log')

class BraCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print("bra")
bra_cb = BraCallback()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[chkpt, csvlog, bra_cb],
    verbose=2
)

In [ ]:
model.load_weights('best_unet.h5')
res = model.evaluate(X_val, y_val, batch_size=BATCH_SIZE, verbose=2)
print(f"VAL →  loss: {res[0]:.4f}   dice: {res[1]:.4f}   acc: {res[2]:.4f}")